# Part II: Gensim LDA

In [1]:
# run functions notbook
%run ../functions/gensim_functions.ipynb

### Load Processed Texts

In [2]:
file_path = "../../data/train_clean.txt"
        
processed_ngrams = load_processed_text(file_path)
print(processed_ngrams[:1])
print(len(processed_ngrams))

[['buy', 'guess', 'flow', 'heavy', 'sort', 'thing', 'back', 'tampon']]
24510


### Baseline Model

In [3]:
# Create Dictionary
id2word = corpora.Dictionary(processed_ngrams)

# Create Corpus
texts = processed_ngrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [4]:
%%time
# LDA Model
model = models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=20,
    random_state=42,
    chunksize=1000,
    passes=10,
    iterations=50,
    update_every = 1,
    alpha='auto',
    eta='auto',  
    eval_every=None # helps to train faster
)

# Compute Coherence Score
coherence_model = models.CoherenceModel(model=model, texts=processed_ngrams, dictionary=id2word, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print('\nCoherence Score:', coherence_score)


Coherence Score: 0.5294372097749029
Wall time: 1min 4s


In [5]:
%%time
# Extract top 50 words for each topic and create texts representing top words for each topic
top_words_texts = [[word for word, _ in model.show_topic(i, topn=50)] for i in range(20)]
top_words_texts

# Compute Coherence Score using top 50 words in each topics
coherence_model_top_words = models.CoherenceModel(topics=top_words_texts, texts=texts, dictionary=id2word, coherence='c_v')
coherence_score_top_words = coherence_model_top_words.get_coherence()
print('\nCoherence Score (Top Words):', coherence_score_top_words)


Coherence Score (Top Words): 0.5294372097749029
Wall time: 12.2 s


### Tune LDA

In [10]:
# Seeds Generation
seeds = generate_random_seeds(3)
seeds

[8558, 3883, 7967]

In [11]:
# Params Grid
num_topics = [10, 20, 50]

grid = {'alpha': ['symmetric','asymmetric','auto', 0.1, 0.01],  
        'eta': ['symmetric', 'auto', 0.1, 0.01]}

In [12]:
df = tune_lda_iter(texts, corpus, id2word, num_topics, seeds, grid)


Total progress:  36%|██████████████████████                                       | 65/180 [1:05:01<2:05:48, 65.64s/it]


Total progress:  73%|███████████████████████████████████████████▋                | 131/180 [2:20:48<1:09:30, 85.12s/it]


Total progress: 100%|██████████████████████████████████████████████████████████████| 180/180 [3:29:41<00:00, 69.90s/it]


In [13]:
df

,num_topics,seed,score,alpha,eta,tests
0,10,8558,0.626402,asymmetric,0.01,"{0: {'top 5 sample': ['problem', 'man', 'body'..."
1,10,3883,0.580637,asymmetric,symmetric,"{0: {'top 5 sample': ['make', 'formula', 'cup'..."
2,10,7967,0.625912,auto,symmetric,"{0: {'top 5 sample': ['wear', 'buy', 'hour', '..."
3,20,8558,0.559939,0.01,0.1,"{0: {'top 5 sample': ['mother', 'live', 'murde..."
4,20,3883,0.569342,0.1,0.01,"{0: {'top 5 sample': ['side', 'weird', 'belly'..."
5,20,7967,0.562992,0.01,0.1,"{0: {'top 5 sample': ['overnight', 'bother', '..."
6,50,8558,0.490047,0.01,0.1,"{0: {'top 5 sample': ['treatment', 'physical',..."
7,50,3883,0.472989,asymmetric,0.1,"{0: {'top 5 sample': ['feel', 'wear', 'pad', '..."
8,50,7967,0.495381,0.01,0.1,"{0: {'top 5 sample': ['push', 'wear', 'start',..."


### Save Results

In [ ]:
path ='../../res/gensim_test.csv'

df_to_csv(df, path)